# Load data

In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import seaborn as sns
import matplotlib as mpl

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt

# Configuration

In [ ]:
# Korean font
from matplotlib import font_manager, rc
try:
    font_path = "C:/Windows/Fonts/malgun.TTF"
    Kfont = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=Kfont)
except:
    pass

# Fix minus presentation
mpl.rcParams['axes.unicode_minus'] = False

# Custom functions

In [ ]:
def to_datetime_format(df:pd.DataFrame, col_nm:str='ymdhm', as_index=False) -> pd.DataFrame:
    df[col_nm] = pd.to_datetime(df[col_nm], infer_datetime_format=True) #format = "%Y-%m-%d %H%M")
    df = df.sort_values(by=col_nm, ascending=True)
    if as_index:
        df.set_index(col_nm, inplace=True)
    
    return df 

# Load data

### Data description
Data  

├ Water Data

│ ├ data_2012.csv

│   ├ ymdhm : 년월일시분

│   ├ swl : 팔당댐 현재수위 (단위: El.m)

│   ├ inf : 팔당댐 유입량 (단위: m^3/s)

│   ├ sfw : 팔당댐 저수량 (단위: 만m^3)

│   ├ ecpc : 팔당댐 공용량 (단위: 백만m^3)

│   ├ tototf : 총 방류량 (단위: m^3/s)

│   ├ tide_level : 강화대교 조위 (단위: cm)

│   ├ wl_1018662 : 청담대교 수위 (단위: cm)

│   ├ fw_1018662 : 청담대교 유량 (단위: m^3/s)

│   ├ wl_1018680 : 잠수교 수위 (단위: cm)

│   ├ fw_1018680 : 잠수교 유량 (단위: m^3/s)

│   ├ wl_1018683 : 한강대교 수위 (단위: cm)

│   ├ fw_1018683 : 한강대교 유량 (단위: m^3/s)

│   ├ wl_1019630 : 행주대교 수위 (단위: cm)

│   └ fw_1019630 : 행주대교 유량 (단위: m^3/s)

│ ├ data_2013.csv

…

└ └ data_2022.csv

└ RainFall Data

│ ├ rf_2012.csv

│   ├ YMDHM : 년월일시분

│   ├ rf_10184100 : 대곡교 강수량

│   ├ rf_10184110 : 진관교 강수량

│   └ rf_10184140 : 송정동 강수량

│ ├ rf_2013.csv

…

└ └ rf_2022.csv

### Load sample submission data

In [ ]:
# Load sample submission data
df_smp_subm = pd.read_csv('data/sample_submission.csv')
df_smp_subm

# Get target columns
tgt_col = df_smp_subm.columns[1:]
tgt_col

### Load water data

In [ ]:
# Get water level data list
wl_data_list = sorted(glob("data/water_data/*.csv"))
wl_data_list

In [ ]:
# check water level data
pd.read_csv(wl_data_list[0]).info()

In [ ]:
# Check most recent water level data
pd.read_csv(wl_data_list[-1])

In [ ]:
# Concat whole water level data
df_wl_all = pd.concat([to_datetime_format(pd.read_csv(path), as_index=True) for path in wl_data_list], axis=0)

### Load rainfall data

In [ ]:
# Get rainfall data list
rf_data_list = sorted(glob("data/rf_data/*.csv"))
rf_data_list

In [ ]:
# check rainfall data
pd.read_csv(rf_data_list[0]).info()

In [ ]:
# Check most recent rainfall data
pd.read_csv(rf_data_list[-1])

In [ ]:
# Concat whole rainfall data
df_rf_all = pd.concat([to_datetime_format(pd.read_csv(path), as_index=True) for path in rf_data_list], axis=0)

# Preprocessing & Visualization

### Simple missing check for water level data

In [ ]:
# Replace value from 0 to None in the target columns
df_wl_all[tgt_col] = df_wl_all[tgt_col].replace({0:None})

# Divide water data by date
df_wl = df_wl_all[df_wl_all.index < pd.to_datetime('2022-06-01')]
df_wl_fut = df_wl_all[df_wl_all.index >= pd.to_datetime('2022-06-01')]

# Get target columns in the train time line
df_tgt_past = df_wl[tgt_col]

In [ ]:
# Water level data except test time missing check
msno.matrix(df_wl)
df_wl.info()

In [ ]:
# Water level future dataset missing check
msno.matrix(df_wl_fut)
df_wl_fut.info()

### Simple missing check for rainfall data

In [ ]:
# Divide rainfall data by date
df_rf = df_rf_all[df_rf_all.index < pd.to_datetime('2022-06-01')]
df_rf_fut = df_rf_all[df_rf_all.index >= pd.to_datetime('2022-06-01')]

In [ ]:
# Rainfall data except test time missing check
msno.matrix(df_rf)
df_rf.info()

In [ ]:
# Rainfall future dataset missing check
msno.matrix(df_rf_fut)
df_rf_fut.info()

### Rainfall data analysis

In [ ]:
# plot - total rainfall heatmap
fig_tot_rf_heat, ax_tot_rf_heat = plt.subplots(1, 1, figsize=(12, 9))

sns.heatmap(
    df_rf_all,
    cmap='RdBu',
)

In [ ]:
# plot - rainfall by lineplot
df_rf_all.plot(
    kind='line',
    subplots=True,
    figsize=(20, 12),
)

In [ ]:
# Plot - Rain fall in June and July
df_rf_june_july = df_rf_all[(df_rf_all.index.month==6) | (df_rf_all.index.month==7)]
df_rf_june_july['year'] = df_rf_june_july.index.year
df_rf_june_july['month'] = df_rf_june_july.index.month
df_rf_june_july.index = np.arange(len(df_rf_june_july.index))
df_rf_june_july.groupby(by=['year', 'month']).mean().plot(
    kind='barh',
    figsize=(10, 10),
)

### Target analysis

In [ ]:
# Past target total by line plot
df_tgt_past.plot(
    figsize=(20, 8),
)

In [ ]:
# Mean target values by year and month
df_tgt_past_june_july = df_tgt_past[(df_tgt_past.index.month==6) | (df_tgt_past.index.month==7)]
df_tgt_past_june_july.plot(
    kind='line',
    figsize=(20, 10),
)

In [ ]:
# Mean target values by year and month
df_tgt_past_june_july = df_tgt_past[(df_tgt_past.index.month==6) | (df_tgt_past.index.month==7)]
df_tgt_past_june_july['year'] = df_tgt_past_june_july.index.year
df_tgt_past_june_july['month'] = df_tgt_past_june_july.index.month

df_tgt_past_june_july.groupby(['year', 'month']).mean().plot(
    kind='line',
    figsize=(20, 10),
)

In [ ]:
a = df_tgt_past.resample(rule='5D', kind='period').mean()
a.plot(figsize=(12, 12))

### Simple analysis of columns that are related to paldang dam in water level 

In [ ]:
# Select columns in water level data related to paldang dam
paldang_cols = df_wl_all.columns[:6]
df_wl_pd = df_wl_all[paldang_cols]
msno.matrix(df_wl_pd)
df_wl_pd.info()

In [ ]:
# plot - simple line of paldang related features
df_wl_pd.plot(
    kind='line',
    figsize=(16, 12),
    subplots=True,
)

In [ ]:
# plot - line plot of all water level data
df_wl_all.plot(
    kind='line',
    subplots=True,
    figsize=(16, 20),
    grid=True,
)

### Analysis water level data of each year seperately 

In [ ]:
# Select columns that are closely related to the target columns
tgt_rt_col = df_wl_all.columns[-8:]
tgt_rt_col

In [ ]:
# Get each water level data seperately
df_wl_each = [to_datetime_format(pd.read_csv(path), as_index=True) for path in wl_data_list]
wl_data_list

In [ ]:
# plot - line plot and null check for each water level data
for yr, df in zip(np.arange(2012, 2023), df_wl_each):
    df.plot(
        kind='line',
        subplots=True,
        figsize=(16, 20),
        title=yr,
    )
    msno.matrix(df)
    print('\n', yr, '\n', (df==0).sum())

In [ ]:
# plot - line plot and null check for columns that are closely related to target columns in water level data
for yr, df in zip(np.arange(2012, 2023), df_wl_each):
    df = df[tgt_rt_col]
    df.plot(
        kind='line',
        subplots=True,
        figsize=(16, 20),
        title=yr,
    )
    msno.matrix(df)
    print('\n', yr, '\n', (df==0).sum())

In [ ]:
# plot - line plot and null check for columns that are closely related to target columns in water level data
# Restrict date from May to July
for yr, df in zip(np.arange(2012, 2023), df_wl_each):
    df = df[tgt_rt_col]
    df = df[(df.index.month==5) | (df.index.month==6) | (df.index.month==7)]
    df.plot(
        kind='line',
        subplots=True,
        figsize=(16, 20),
        title=yr,
    )